In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sodapy import Socrata

"""
This notebook is designed to develop the relationship between mortality data due to cardiovascular desease
and defibrilator placements (medical and non-medical). Of course, AED will be used before death, but this data
could account for potential areas that have a higher degree of mortality due to cardiovascular desease. 
Analysis will be done considering comarcal geographical distributions, which is the highest resolution data
we have related to mortality rates.
Of course, AEDs can be used in other medical related deseases, but we will specially focus on cardiovascular 
due to the direct relationship.
"""



"""
The first step is to load the available data into the df and clean it. In dades.ipnyb the mortality data, 
defibrillator and hospital data was cleaned.
"""
client = Socrata("analisi.transparenciacatalunya.cat", None)
desfibriladors = client.get_all("wpud-ukyg")
hospitals = client.get_all("8gmd-gz7i")
df_desfibriladors = pd.DataFrame.from_records(desfibriladors) 
df_hospitals = pd.DataFrame.from_records(hospitals)

df_hospitals = df_hospitals[df_hospitals["categoria"].str.contains(r"Salut\|Centres", na=False)]
df_hospitals = df_hospitals.drop_duplicates(subset='idequipament', keep='first')


df_desfibriladors = df_desfibriladors.drop_duplicates(subset='numero_serie', keep='first')


url = "https://www.idescat.cat/serveis/consultes/ca/censph_10_mun_2024.csv"
df_cens = pd.read_csv(url, sep=";", encoding="utf-8")


df_mortality = pd.read_csv("./Data/mortality.csv", sep=",", encoding="utf-8-sig")


/var/folders/_r/lv31y_xx5hzfyhbw22ddz7gr0000gn/T/ipykernel_23697/1405617251.py:36: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cens = pd.read_csv(url, sep=";", encoding="utf-8")


In [25]:
"""
We read both geopandas documents and normalise the names and change some column names so we work with
the same identifiers all around.
"""
import unicodedata

"""
This makes all the names similar so it is easier to compare them
"""
def normalize(text):
    if isinstance(text, str):
        text = text.lower().strip()
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = text.replace('-', ' ').replace("'", "").replace("’", "")
        return text
    return text

data_dir = "../carto/"

municipis = gpd.read_file(data_dir + "municipis.json")
municipis = municipis.rename(columns={'CODIMUNI': 'codi_municipi', 'NOMMUNI':'municipi', 'NOMCOMAR': 'comarca'})
municipis = municipis[['codi_municipi', 'municipi', 'comarca', 'geometry']]
municipis

comarques = gpd.read_file(data_dir + "comarca.json")

comarques = comarques[["NOMCOMAR","geometry"]].rename(columns={"NOMCOMAR":"comarca"})
comarques['comarca'] = comarques['comarca'].apply(normalize)

comarca_map = pd.read_csv("./Data/mpiscatalunya.csv", sep=";", encoding="utf-8")
comarca_map = comarca_map.rename(columns={'Codi': 'codi_municipi', 'Nom':'municipi',
                                          'Nom comarca': 'comarca'})


In [26]:
"""
Now we want to count how many hospitals/AED we have per comarca, and create a df that contains these values
and merge it inot a gpd to have the correct idea.
We also add a column with the mortality per 1000 inh for men/women/total due to cardivascular desease
for each of the comarques.
"""

df_hospi = df_hospitals[['codi_municipi']]
df_desfi = df_desfibriladors[['codi_municipi']]

defi_count = df_desfi.groupby('codi_municipi').size().reset_index(name='num_defi')
hospi_count = df_hospi.groupby('codi_municipi').size().reset_index(name='num_hospi')


In [31]:
"""
Normalise municipal data to make sure all have the same codes
It fixes the municipal codes correctly between datasets.
"""
def fix_code(x):
    x = str(x)  # ensure string
    # Add leading zero if first digit is 8 and length < 6
    if x.startswith('8') and len(x) < 6:
        x = '0' + x
    # Truncate last digit if length is 6
    if len(x) == 6:
        x = x[:-1]
    return x


defi_count['codi_municipi'] = defi_count['codi_municipi'].apply(fix_code)
hospi_count['codi_municipi'] = hospi_count['codi_municipi'].apply(fix_code)
municipis['codi_municipi'] = municipis['codi_municipi'].apply(fix_code)
comarca_map['codi_municipi'] = comarca_map['codi_municipi'].apply(fix_code)

# Filter rows where edat == "total", "homes", "dones"
def population_df(df, sexe):
    
    df = df[(df["edat"] == "total") & (df["sexe"] == sexe)].copy()

    # filter out total population. The max value for each case is below 3,000,000
    df = df[df["valor"] <= 3000000].copy()

    df = df.reset_index(drop=True)
    df = df.rename(columns={"valor": "pob"})

    df_out = df[["geo", "pob"]].copy()
    df_out = df_out.rename(columns={'geo': 'codi_municipi'})
    df_out["codi_municipi"] = df_out["codi_municipi"].apply(fix_code)

    return df_out



df_pop = {
    "total": population_df(df_cens, "total"),
    "men": population_df(df_cens, "homes"), 
    "women": population_df(df_cens, "dones")
}


In [42]:
import numpy as np
from shapely.validation import make_valid
"""
After all the data cleaning, we finally got to where we want to be, a data set that has municipi and comarca and the amount of
defibrillators and medical centers in each of these places.

Also, we sum the polygons for each of our comarcas (2022) because the mortality data is for that year
and lluçanes did not exist. This allows us to treat our data uniformly.
"""
comarca_totals={}

for key, df in df_pop.items():
    total = municipis.merge(defi_count, on='codi_municipi', how='left')
    total = total.merge(hospi_count, on='codi_municipi', how='left')
    total = total.merge(df, on='codi_municipi', how='left')
    
    total = total.fillna(0)
    total = total.replace([np.inf, -np.inf], 0)
    total = total.drop(columns=['comarca'])

    #We assume that if we take into account the sex, then the resources are divided by 2 (as a uniforma dist).
    #Of course, the poob per municipality is not 50/50, but regarding resources we won't divide differently.
    if key != "total":
        total["num_defi"] = total["num_defi"]/2
        total["num_hospi"] = total["num_hospi"]/2
    
    total = total.merge(
        comarca_map[['codi_municipi', 'comarca']],
        on='codi_municipi',
        how='left'
    )
    
    total['geometry'] = total['geometry'].apply(make_valid)
    total = total.set_crs("EPSG:4326", allow_override=True)
    
    comarca_totals[key] = total.groupby('comarca')[['num_defi', 'num_hospi', 'pob']].sum().reset_index()
    
    gdf_comarca = total.dissolve(by='comarca', as_index=False)
    gdf_comarca = gdf_comarca[['comarca', 'geometry']]
    comarca_totals[key] = comarca_totals[key].merge(gdf_comarca, on='comarca', how='left')
    comarca_totals[key] = gpd.GeoDataFrame(
        comarca_totals[key],
        geometry='geometry',  # tell GeoPandas which column is geometry
        crs=gdf_comarca.crs  # use the same CRS as your dissolved gdf
    )
    
    numeric_cols = ['num_defi', 'num_hospi', 'pob']
    
    for col in numeric_cols:
        comarca_totals[key][col] = pd.to_numeric(comarca_totals[key][col], errors='coerce').fillna(0)


In [43]:
"""
Now comes the cleaning and doing with the medical data. we can select a df when choosing one of 
keys, this allows us to save the whole cleaned df in a compact way. We select one of the death
causes, like Circ (circulatory problems) or Tot (Total). Since these values are death rates, we
should multiply them by the probability that a AED has been used...

From data in OHSCAR, we have that cardiac arrests constitute 45.8/100,000 inhabitants for 2021 in
Catalunya. This means, that a rough estimate is that comarca_pob * cardiac_arrest gives us the
desired data. We can compare this with Circ or other death causes. These cardiac arrests values are
out of hospital cardiac arrests, so work better for DEAs. 
To make it more comarcal-like, assume that every cardiac arrest would result in death if untreated,
then we calculate the percentatge of cardiac_arrests/total_circ_death, which gives us a % where DEAS 
can be used. Then, we scale to each comarcal circ_death.
"""
df_cardio = df_mortality[df_mortality['Cause'].str.contains('Circ', case=False, na=False)]

sexes = ["Total", "Dones", "Homes"]

dfs = {}
for name in sexes:
    df = (df_cardio[df_cardio['Sexe'] == name]
          .drop(columns=["Sexe", "Cause"])
          .rename(columns={'Comarca':'comarca'})
          .reset_index(drop=True))
    df['comarca'] = df['comarca'].apply(normalize)
    dfs[name] = df

mortality_totals = dfs["Total"]
mortality_women  = dfs["Dones"]
mortality_men    = dfs["Homes"]


In [46]:
"""
We compute the data but now based on comarca and we eliminate llucanes comarca, because it is newer
than older df. We have to take into account that this comarca is small.

Then, we compute the cardiac which essetially is the percentatge of circ deaths per comarca that 
could be caused by a cardia arrest. if we take into account that 45.8/100,000 cases in catalunya,
this is approx: 8.012.231 population gives 3.670 cardiac arrest, which represent a 22.38% of total
circ deaths (16.400). We take this value to calcualte cardiac rel
"""
for key, df in comarca_totals.items():
    comarca_totals[key]['comarca'] = comarca_totals[key]['comarca'].apply(normalize)


mortality_dfs = {
    "total": mortality_totals,
    "women": mortality_women,
    "men": mortality_men
}

w_a = {'1-13': 0.0037, '13-50': 0.124, '50-75': 0.404, '75+': 0.469}
age_map = {
    '< 1':'1-13','1-4':'1-13','5-14':'1-13','15-24':'13-50','25-34':'13-50',
    '35-44':'13-50','45-54':'50-75','55-64':'50-75','65-74':'50-75',
    '75-84':'75+','>84':'75+'
}

df_comarques = {}

for key, df_mort in mortality_dfs.items():
    df_comarques[key] = comarca_totals[key].merge(
        df_mort,
        on='comarca',
        how='left'
    )
    #Raw computation of OHCA based only on total cardiac mortality and no age factors.
    df_comarques[key]["cardiac rel"] = df_comarques[key]["Total rel"]*(3670/16400)
    
    # Multiply each age group by OHCA weight, This gives an edge weighted OHCA
    df = df_comarques[key].copy()
    ohca_signal = []
    for age_col in age_map.keys():
        ohca_signal.append(df[age_col] * w_a[age_map[age_col]])
    df['ohca_signal'] = sum(ohca_signal)
    
    # Anchor to total observed OHCA
    k = 3670 / df['ohca_signal'].sum()
    df['est_ohca'] = df['ohca_signal'] * k
    df_comarques[key]['ohca rel'] = df['est_ohca'] / df['pob'] * 100000

    cols_to_keep = [
        'comarca',
        'num_defi',
        'num_hospi',
        'pob',
        'geometry',
        'Total rel',
        'cardiac rel',
        'ohca rel'
    ]
    df_comarques[key] = df_comarques[key][cols_to_keep].copy()
    
df_comarques_total = df_comarques["total"]
df_comarques_women = df_comarques["women"]
df_comarques_men   = df_comarques["men"]

df_comarques_men


,comarca,num_defi,num_hospi,pob,geometry,Total rel,cardiac rel,ohca rel
0,alt camp,32.0,11.0,23639,"POLYGON ((1.23383 41.20393, 1.23383 41.20472, ...",174.30,39.004939,82.165531
1,alt emporda,141.5,25.5,74644,"MULTIPOLYGON (((2.997 42.13316, 2.99723 42.132...",207.84,46.510537,99.082129
2,alt penedes,118.0,12.0,57168,"POLYGON ((1.69316 41.2768, 1.69315 41.27542, 1...",193.88,43.386561,91.748551
3,alt urgell,26.5,9.0,10632,"MULTIPOLYGON (((1.32159 41.98664, 1.32235 41.9...",323.84,72.469073,153.900962
4,alta ribagorca,7.5,5.0,2107,"POLYGON ((0.85715 42.45393, 0.87114 42.44601, ...",142.72,31.937951,64.559949
5,anoia,221.0,11.0,64829,"POLYGON ((1.60696 41.51295, 1.60717 41.51292, ...",212.81,47.622726,99.709040
6,aran,15.5,3.5,5397,"POLYGON ((0.68006 42.72327, 0.68002 42.72329, ...",130.09,29.111604,58.382484
7,bages,129.0,13.5,92073,"POLYGON ((1.81103 41.60503, 1.81002 41.6048, 1...",232.96,52.131902,110.135615
8,baix camp,109.0,14.0,100871,"MULTIPOLYGON (((0.93153 40.99126, 0.93134 40.9...",188.65,42.216189,88.368473
9,baix ebre,67.0,11.5,41884,"MULTIPOLYGON (((0.44356 40.73713, 0.44377 40.7...",255.00,57.064024,121.094197


In [47]:
"""
We compute the AED coverage per 100.000k inhabitants. Important:
· Consider only AED locations, very different of being in a hospital from behind at home
· Cardiac arrest data was for Out of Hospital, so much better rationale.
"""
def coverage_calculation(df, pop = "pob", num = "num_defi", cov = "coverage_desfi"):
    df[cov] = np.where(
        df[pop] > 0,
        (df[num]) / df[pop] * 100000,
        0
    )
    return df


for key, df in df_comarques.items():

    # Convert columns to numeric
    df['num_defi']   = pd.to_numeric(df['num_defi'], errors='coerce').fillna(0)
    df['num_hospi']  = pd.to_numeric(df['num_hospi'], errors='coerce').fillna(0)
    df['pob']        = pd.to_numeric(df['pob'], errors='coerce').fillna(0)
    df['Total rel']  = pd.to_numeric(df['Total rel'], errors='coerce').fillna(0)

    # Compute coverage per 100k
    df = coverage_calculation(df, pop = "pob", num = "num_defi", cov = "coverage_desfi")
    df = coverage_calculation(df, pop = "pob", num = "num_hospi", cov = "coverage_hospi")
    df["total_coverage"] = df["coverage_hospi"] + df["coverage_desfi"]
    df_comarques[key] = df



In [48]:
"""
We compute the normalised values for each column per comarca. This value is first centralised with the mean,
and then normalised to a maximum. This way, all variables have no units and are better plotted one with 
the other. 

The zero mask is no longer needed. 
"""

def normalise_zscore(df_comarques):
    for key, df in df_mort_com.items():
    
        zeros = df[cause] == 0
        non_zeros = ~zeros
    
        # Mean only over non-zero values
        mean_cause = df.loc[non_zeros, cause].mean()
        std_cause  = df.loc[non_zeros, cause].std()
    
        df[f'{cause} norm'] = 0.0
        if std_cause != 0:
            df.loc[non_zeros, f'{cause} norm'] = (df.loc[non_zeros, cause] - mean_cause) / std_cause
    
        # ---- COVERAGE NORM ----
        mean_cov = df['coverage_per_100k'].mean()
        std_cov  = df['coverage_per_100k'].std()
    
        df['coverage_norm'] = 0.0
        if std_cov != 0:
            df['coverage_norm'] = (df['coverage_per_100k'] - mean_cov) / std_cov
    
        df_comarques[key] = df
    return df_comarques



def normalize_percentile_trim(df, col, zero_mask=None, low_pct=0, high_pct=100):

    if zero_mask is None:
        zero_mask = (df[col] == 0)

    nonzero_mask = ~np.asarray(zero_mask)
    values = df.loc[nonzero_mask, col].astype(float).values

    if values.size < 3:
        raise ValueError("Need at least 3 non-zero values for percentile trimming.")

    low = np.percentile(values, low_pct)
    high = np.percentile(values, high_pct)
    if low == high:
        raise ValueError("Percentile bounds collapsed to a single value. Choose different percentiles.")

    inside_mask = (values >= low) & (values <= high)
    if not inside_mask.any():
        raise ValueError("No values lie inside the chosen percentile bounds.")

    mean_inside = values[inside_mask].mean()
    centered = values - mean_inside
    low_adj = low - mean_inside
    high_adj = high - mean_inside

    scale = max(abs(low_adj), abs(high_adj))
    if scale == 0:
        scale = 1e-9
    normalized = np.clip(centered / scale, -1, 1)
    norm_col = f"{col}_norm"
    df[norm_col] = 0.0
    df.loc[nonzero_mask, norm_col] = normalized

    return df

def normalise_maxabs(df_grouped, cause='cardiac rel', ohca = 'ohca rel', desfi='coverage_desfi', hospi = 'coverage_hospi', total = 'total_coverage', pop='pob'):
    for key, df in df_grouped.items():
        
        zeros = df[cause] == 0
        non_zeros = ~zeros

        df = normalize_percentile_trim(df, cause , zero_mask = zeros)
        df = normalize_percentile_trim(df, ohca , zero_mask = zeros)
        df = normalize_percentile_trim(df, pop , zero_mask = zeros, high_pct = 86)
        df = normalize_percentile_trim(df, desfi, zero_mask = zeros, high_pct = 95)
        df = normalize_percentile_trim(df, hospi, zero_mask = zeros, high_pct = 95)
        df = normalize_percentile_trim(df, total, zero_mask = zeros, high_pct = 95)
        df_grouped[key]=df
        
    return df_grouped


df_comarques = normalise_maxabs(df_comarques)


df_comarques_women



,comarca,num_defi,num_hospi,pob,geometry,Total rel,cardiac rel,ohca rel,coverage_desfi,coverage_hospi,total_coverage,cardiac rel_norm,ohca rel_norm,pob_norm,coverage_desfi_norm,coverage_hospi_norm,total_coverage_norm
0,alt camp,32.0,11.0,22749,"POLYGON ((1.23383 41.20393, 1.23383 41.20472, ...",211.56,47.343000,88.699508,140.665524,48.353774,189.019298,-0.239685,-0.252888,-0.180892,-0.330873,0.123788,-0.167888
1,alt emporda,141.5,25.5,74088,"MULTIPOLYGON (((2.997 42.13316, 2.99723 42.132...",214.64,48.032244,90.402495,190.989094,34.418529,225.407623,-0.215993,-0.222232,0.473061,-0.006471,-0.030234,-0.016612
2,alt penedes,118.0,12.0,57021,"POLYGON ((1.69316 41.2768, 1.69315 41.27542, 1...",183.66,41.099524,77.459470,206.941302,21.044878,227.986181,-0.454303,-0.455225,0.255663,0.096362,-0.178049,-0.005893
3,alt urgell,26.5,9.0,10496,"MULTIPOLYGON (((1.32159 41.98664, 1.32235 41.9...",327.24,73.229927,137.528496,252.477134,85.746951,338.224085,0.650169,0.626105,-0.336970,0.389901,0.537083,0.452395
4,alta ribagorca,7.5,5.0,1933,"POLYGON ((0.85715 42.45393, 0.87114 42.44601, ...",207.47,46.427738,89.311068,387.997931,258.665287,646.663218,-0.271147,-0.241879,-0.446045,1.000000,1.000000,1.000000
5,anoia,221.0,11.0,63603,"POLYGON ((1.60696 41.51295, 1.60717 41.51292, ...",203.99,45.648982,86.596056,347.467887,17.294782,364.762668,-0.297917,-0.290753,0.339504,1.000000,-0.219498,0.562723
6,aran,15.5,3.5,5148,"POLYGON ((0.68006 42.72327, 0.68002 42.72329, ...",155.25,34.741921,67.070045,301.087801,67.987568,369.075369,-0.672843,-0.642250,-0.405093,0.703261,0.340794,0.580652
7,bages,129.0,13.5,93279,"POLYGON ((1.81103 41.60503, 1.81002 41.6048, 1...",285.32,63.849049,121.459473,138.294793,14.472711,152.767504,0.327704,0.336839,0.717515,-0.346156,-0.250689,-0.318596
8,baix camp,109.0,14.0,103587,"MULTIPOLYGON (((0.93153 40.99126, 0.93134 40.9...",177.87,39.803835,73.883946,105.225559,13.515209,118.740769,-0.498842,-0.519590,0.848818,-0.559331,-0.261272,-0.460054
9,baix ebre,67.0,11.5,40515,"MULTIPOLYGON (((0.44356 40.73713, 0.44377 40.7...",317.96,71.153244,134.731033,165.370850,28.384549,193.755399,0.578784,0.575747,0.045410,-0.171615,-0.096926,-0.148199


In [50]:
"""
We save our files, so we can later import them and plot multiple things.
"""
df_comarques_total.to_file("./Data/df_comarques_total.gpkg", driver="GPKG")
df_comarques_men.to_file("./Data/df_comarques_men.gpkg", driver="GPKG")
df_comarques_women.to_file("./Data/df_comarques_women.gpkg", driver="GPKG")
